In [ ]:
import functools
import math
import time
from typing import Tuple

import jax
import jax.numpy as jnp

try:
    import brax
except:
    !pip install git+https://github.com/google/brax.git@v0.9.2 |tail -n 1
    import brax

try:
    import flax
except:
    !pip install --no-deps git+https://github.com/google/flax.git@v0.7.4 |tail -n 1
    import flax

try:
    import chex
except:
    !pip install --no-deps git+https://github.com/deepmind/chex.git@v0.1.83 |tail -n 1
    import chex

try:
    import jumanji
except:
    !pip install "jumanji==0.3.1"
    import jumanji

try:
    import haiku
except:
    !pip install git+https://github.com/deepmind/dm-haiku.git@v0.0.5 |tail -n 1
    import haiku

try:
    import qdax
except:
    !pip install --no-deps git+https://github.com/adaptive-intelligent-robotics/QDax@main |tail -n 1
    import qdax

import matplotlib.pyplot as plt
from brax.v1.io import html
from IPython.display import HTML
from tqdm import tqdm

from qdax import environments
from qdax.baselines.pbt import PBTTrainingState
from qdax.baselines.td3_pbt import PBTTD3, PBTTD3Config
from qdax.core.containers.mapelites_repertoire import compute_cvt_centroids
from qdax.core.emitters.pbt_me_emitter import PBTEmitter, PBTEmitterConfig
from qdax.core.emitters.pbt_variation_operators import td3_pbt_variation_fn
from qdax.core.distributed_map_elites import DistributedMAPElites
from qdax.types import RNGKey
from qdax.utils.metrics import default_qd_metrics
from qdax.utils.plotting import plot_2d_map_elites_repertoire, plot_map_elites_results

In [ ]:
jax.config.update("jax_platform_name", "cpu")

In [ ]:
devices = jax.devices("tpu")
num_devices = len(devices)
print(f"Detected the following {num_devices} device(s): {devices}")

In [ ]:
env_name = "anttrap"

seed = 0

# TD3 config
episode_length = 1000
batch_size = 256
policy_delay = 2
soft_tau_update = 0.005
critic_hidden_layer_size = (256, 256)
policy_hidden_layer_size = (256, 256)

# Emitter config
buffer_size = 100000
pg_population_size_per_device = 10
ga_population_size_per_device = 10
num_training_steps = 5000
env_batch_size = 250
grad_updates_per_step = 1.0
iso_sigma = 0.005
line_sigma = 0.05

fraction_best_to_replace_from = 0.1
fraction_to_replace_from_best = 0.2
fraction_to_replace_from_samples = 0.4
# this fraction is used only for transfer between devices
fraction_sort_exchange = 0.1

eval_env_batch_size = 1

# MAP-Elites config
num_init_cvt_samples = 50000
num_centroids = 1024
log_period = 1
num_iterations = 20
save_repertoire_freq = 5
# num_iterations = 450
# save_repertoire_freq = 15

In [ ]:
%%time
# Initialize environments
env = environments.create(
    env_name=env_name,
    batch_size=env_batch_size * pg_population_size_per_device,
    episode_length=episode_length,
    auto_reset=True,
)

eval_env = environments.create(
    env_name=env_name,
    batch_size=eval_env_batch_size,
    episode_length=episode_length,
    auto_reset=True,
)

In [ ]:
min_bd, max_bd = env.behavior_descriptor_limits

In [ ]:
%%time
key = jax.random.PRNGKey(seed)
key, subkey = jax.random.split(key)
env_states = jax.jit(env.reset)(rng=subkey)
eval_env_first_states = jax.jit(eval_env.reset)(rng=subkey)

In [ ]:
# get agent
config = PBTTD3Config(
    episode_length=episode_length,
    batch_size=batch_size,
    policy_delay=policy_delay,
    soft_tau_update=soft_tau_update,
    critic_hidden_layer_size=critic_hidden_layer_size,
    policy_hidden_layer_size=policy_hidden_layer_size,
)

agent = PBTTD3(config=config, action_size=env.action_size)

In [ ]:
# init emitter
emitter_config = PBTEmitterConfig(
    buffer_size=buffer_size,
    num_training_iterations=num_training_steps // env_batch_size,
    env_batch_size=env_batch_size,
    grad_updates_per_step=grad_updates_per_step,
    pg_population_size_per_device=pg_population_size_per_device,
    ga_population_size_per_device=ga_population_size_per_device,
    num_devices=num_devices,
    fraction_best_to_replace_from=fraction_best_to_replace_from,
    fraction_to_replace_from_best=fraction_to_replace_from_best,
    fraction_to_replace_from_samples=fraction_to_replace_from_samples,
    fraction_sort_exchange=fraction_sort_exchange,
)

In [ ]:
variation_fn = functools.partial(
    td3_pbt_variation_fn, iso_sigma=iso_sigma, line_sigma=line_sigma
)

In [ ]:
emitter = PBTEmitter(
    pbt_agent=agent,
    config=emitter_config,
    env=env,
    variation_fn=variation_fn,
)

In [ ]:
# get scoring function
bd_extraction_fn = environments.behavior_descriptor_extractor[env_name]
eval_policy = agent.get_eval_qd_fn(eval_env, bd_extraction_fn=bd_extraction_fn)


def scoring_function(genotypes, random_key):
    population_size = jax.tree_leaves(genotypes)[0].shape[0]
    first_states = jax.tree_map(
        lambda x: jnp.expand_dims(x, axis=0), eval_env_first_states
    )
    first_states = jax.tree_map(
        lambda x: jnp.repeat(x, population_size, axis=0), first_states
    )
    population_returns, population_bds, _, _ = eval_policy(genotypes, first_states)
    return population_returns, population_bds, None, random_key

In [ ]:
# Get minimum reward value to make sure qd_score are positive
reward_offset = environments.reward_offset[env_name]

# Define a metrics function
metrics_function = functools.partial(
    default_qd_metrics,
    qd_offset=reward_offset * episode_length,
)

# Get the MAP-Elites algorithm
map_elites = DistributedMAPElites(
    scoring_function=scoring_function,
    emitter=emitter,
    metrics_function=metrics_function,
)

In [ ]:
%%time
centroids, key = compute_cvt_centroids(
    num_descriptors=env.behavior_descriptor_length,
    num_init_cvt_samples=num_init_cvt_samples,
    num_centroids=num_centroids,
    minval=min_bd,
    maxval=max_bd,
    random_key=key,
)

In [ ]:
key, *keys = jax.random.split(key, num=1 + num_devices)
keys = jnp.stack(keys)

In [ ]:
%%time
# get the initial training states and replay buffers
agent_init_fn = agent.get_init_fn(
    population_size=pg_population_size_per_device + ga_population_size_per_device,
    action_size=env.action_size,
    observation_size=env.observation_size,
    buffer_size=buffer_size,
)
keys, training_states, _ = jax.pmap(agent_init_fn, axis_name="p", devices=devices)(keys)

In [ ]:
%%time
# empty optimizers states to avoid too heavy repertories
training_states = jax.pmap(
    jax.vmap(training_states.__class__.empty_optimizers_states),
    axis_name="p",
    devices=devices,
)(training_states)

# initialize map-elites
repertoire, emitter_state, keys = map_elites.get_distributed_init_fn(
    devices=devices, centroids=centroids
)(genotypes=training_states, random_key=keys)

In [ ]:
update_fn = map_elites.get_distributed_update_fn(
    num_iterations=log_period, devices=devices
)

In [ ]:
env_step_multiplier = (
    (pg_population_size_per_device + ga_population_size_per_device)
    * eval_env_batch_size
    * episode_length
    + num_training_steps * pg_population_size_per_device
) * num_devices

In [ ]:
%%time
all_metrics = {}
repertoires = []

for i in tqdm(range(num_iterations // log_period), total=num_iterations // log_period):
    start_time = time.time()

    repertoire, emitter_state, keys, metrics = update_fn(
        repertoire, emitter_state, keys
    )
    metrics_cpu = jax.tree_map(lambda x: jax.device_get(x)[0], metrics)
    timelapse = time.time() - start_time

    # log metrics
    for key, value in metrics_cpu.items():
        # take all values
        if key in all_metrics.keys():
            all_metrics[key] = jnp.concatenate([all_metrics[key], value])
        else:
            all_metrics[key] = value

    if i % save_repertoire_freq == 0:
        repertoires.append(jax.tree_map(lambda x: jax.device_get(x)[0], repertoire))

In [ ]:
env_steps = (jnp.arange(num_iterations * log_period) + 1) * env_step_multiplier

# create the plots and the grid
fig, axes = plot_map_elites_results(
    env_steps=env_steps,
    metrics=all_metrics,
    repertoire=repertoires[-1],
    min_bd=min_bd,
    max_bd=max_bd,
)

In [ ]:
%%time
import math

import matplotlib.pyplot as plt

num_repertoires = len(repertoires)
num_cols = 5

fig, axes = plt.subplots(
    nrows=math.ceil(num_repertoires / num_cols), ncols=num_cols, figsize=(30, 30)
)
for i, repertoire in enumerate(repertoires):

    col_i = i % num_cols
    row_i = i // num_cols

    plot_2d_map_elites_repertoire(
        centroids=centroids,
        # repertoire_fitnesses=repertoire.fitnesses,
        repertoire_fitnesses=jnp.where(
            repertoire.fitnesses > -jnp.inf,
            repertoire.genotypes.expl_noise,
            -jnp.inf * jnp.ones_like(repertoire.fitnesses),
        ),
        minval=min_bd,
        maxval=max_bd,
        ax=axes[row_i, col_i],
    )
    axes[row_i, col_i].set_title(f"Grid after {env_step_multiplier * i} steps")